<a href="https://colab.research.google.com/github/heimtathurs/stable-diffusion-videos/blob/video-generation-test/stable_diffusion_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Videos

This notebook allows you to generate videos by interpolating the latent space of [Stable Diffusion](https://github.com/CompVis/stable-diffusion).

You can either dream up different versions of the same prompt, or morph between different text prompts (with seeds set for each for reproducibility).

If you like this notebook:
- consider giving the [repo a star](https://github.com/nateraw/stable-diffusion-videos) ⭐️
- consider following me on Github [@nateraw](https://github.com/nateraw) 

You can file any issues/feature requests [here](https://github.com/nateraw/stable-diffusion-videos/issues)

Enjoy 🤗

## Setup

In [1]:
%%capture
! pip install stable_diffusion_videos[realesrgan]
! git config --global credential.helper store

## Authenticate with Hugging Face Hub

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


## Run the App 🚀

### Load the Interface

This step will take a couple minutes the first time you run it.

In [ ]:
from stable_diffusion_videos import interface

In [4]:
#@title Connect to Google Drive to Save Outputs

#@markdown If you want to connect Google Drive, click the checkbox below and run this cell. You'll be prompted to authenticate.

#@markdown If you just want to save your outputs in this Colab session, don't worry about this cell

connect_google_drive = True #@param {type:"boolean"}

#@markdown Then, in the interface, use this path as the `output` in the Video tab to save your videos to Google Drive:

#@markdown > /content/gdrive/MyDrive/stable_diffusion_videos


if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Launch

This cell launches a Gradio Interface. Here's how I suggest you use it:

1. Use the "Images" tab to generate images you like.
    - Find two images you want to morph between
    - These images should use the same settings (guidance scale, height, width)
    - Keep track of the seeds/settings you used so you can reproduce them

2. Generate videos using the "Videos" tab
    - Using the images you found from the step above, provide the prompts/seeds you recorded
    - Set the `num_interpolation_steps` - for testing you can use a small number like 3 or 5, but to get great results you'll want to use something larger (60-200 steps). 

💡 **Pro tip** - Click the link that looks like `https://<id-number>.gradio.app` below , and you'll be able to view it in full screen.

In [ ]:
interface.launch(debug=True)

---

## Use `walk` programatically

The other option is to not use the interface, and instead use `walk` programatically. Here's how you would do that...

**You may need to reset runtime before running these cells if you used the interface above**

First we define a helper fn for visualizing videos in colab

In [5]:
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Walk! 🚶‍♀️

In [2]:
from stable_diffusion_videos import StableDiffusionWalkPipeline
from diffusers.schedulers import LMSDiscreteScheduler
import torch

pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    use_auth_token=True,
    torch_dtype=torch.float16,
    revision="fp16",
    scheduler=LMSDiscreteScheduler(
        beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear"
    )
).to("cuda")

OSError: ignored

### Bonus! Music videos... upload an audio file and give it a try

In [ ]:
import random

fps = 60

prompts = [
      '30s retrofuturistic city',
      'golden hour',
      'silhouette in the sun',
      'colorful 60s kaleidoscope',
      'vibrant color flashes',
      'retro poster with emerging flower',
      'retro poster with spaceship launch',
      'retro poster to the stars',
      'spaceship gaining speed',
      'double star system',
      'spaceship flying through the nebula',
      'pillars of creation nebula',
      'ocean planet',
      'underwater city',
      'underwater civilization',
      'bedroom at night',
      'waking up',
]

prompts = list(map(lambda i: 'symmetrical ' + i, prompts))
prompts = [p for p in prompts for _ in range(3)]

seeds = [random.randrange(1, 99999) for i in range(len(prompts))]

print('prompts', prompts)
print('seeds', seeds)

video_path = pipeline.walk(
    prompts=prompts,
    seeds=seeds,
    num_interpolation_steps=fps,
    height=1024,                           # use multiples of 64
    width=576,                            # use multiples of 64
    fps=fps,                              # important to set yourself based on the num_interpolation_steps you defined
    batch_size=1,                         # increase until you go out of memory
    output_dir='./dreams',                # Where images will be saved
    name=None,                            # Subdir of output dir. will be timestamp by default
)
visualize_video_colab(video_path)